In [1]:
import pathlib
import pandas as pd
import shutil
import logging

import sys
sys.path.append("../")
from idrstream.DP_idr import DeepProfilerRun

In [2]:
data_to_process = pd.read_csv("example_files/data_to_process.tsv", sep="\t", index_col=0)
data_to_process

,Plate,Well,Frames,Well Number,Control Type,Original Gene Target,Plate_Map_Name,Gene_Replicate,Site,DNA
0,LT0001_02,A15,43,15,negative control,negative control,LT0001_02_15,1,1,LT0001_02/LT0001_02_15_43.tif
1,LT0001_02,B2,46,26,negative control,negative control,LT0001_02_26,1,1,LT0001_02/LT0001_02_26_46.tif
2,LT0001_02,C15,52,63,negative control,negative control,LT0001_02_63,1,1,LT0001_02/LT0001_02_63_52.tif
3,LT0001_02,D2,31,74,negative control,negative control,LT0001_02_74,1,1,LT0001_02/LT0001_02_74_31.tif
4,LT0001_02,M16,34,304,negative control,negative control,LT0001_02_304,1,1,LT0001_02/LT0001_02_304_34.tif
...,...,...,...,...,...,...,...,...,...,...
3848,LT0603_06,D2,51,74,negative control,negative control,LT0603_06_74,1,1,LT0603_06/LT0603_06_74_51.tif
3849,LT0603_06,M16,52,304,negative control,negative control,LT0603_06_304,1,1,LT0603_06/LT0603_06_304_52.tif
3850,LT0603_06,N3,49,315,negative control,negative control,LT0603_06_315,1,1,LT0603_06/LT0603_06_315_49.tif
3851,LT0603_06,O16,47,352,negative control,negative control,LT0603_06_352,1,1,LT0603_06/LT0603_06_352_47.tif


In [3]:
# path to users home directory
home_dir = pathlib.Path.home()

idr_id = "idr0013"
tmp_dir = pathlib.Path("../tmp/")
final_data_dir = pathlib.Path("example_extracted_features/DP_features")
try:
    shutil.rmtree(tmp_dir)
    # shutil.rmtree(final_data_dir)
except:
    print("No files to remove")

stream = DeepProfilerRun(idr_id, tmp_dir, final_data_dir, log='example_logs/dp_idrstream.log')

In [4]:
aspera_path = pathlib.Path(f"{home_dir}/.aspera/ascli/sdk/ascp")
aspera_key_path = pathlib.Path("example_files/asperaweb_id_dsa.openssh")
screens_path = pathlib.Path("example_files/idr0013-screenA-plates.tsv")
save_dir = pathlib.Path("data/")

stream.init_downloader(aspera_path, aspera_key_path, screens_path)

In [5]:
fiji_path = pathlib.Path(f"{home_dir}/Desktop/Fiji.app")
stream.init_preprocessor(fiji_path)

[INFO] Overriding Leica ROI Reader; identifier: command:de.biovoxxel.utilities.RoiReader; jar: file:/home/roshankern/Desktop/Fiji.app/plugins/Biovoxxel_Plugins-2.5.6.jar


[ERROR] Cannot create plugin: org.scijava.plugins.scripting.javascript.JavaScriptScriptLanguage


In [6]:
nuclei_model_specs = {
            "model_type": "cyto",
            "channels": [0, 0],
            "diameter": 0,
            "flow_threshold": 0.8,
            "cellprob_threshold": 0,
            "remove_edge_masks": True,
        }
stream.init_segmentor(nuclei_model_specs)

>>> GPU activated? 1


In [7]:
config_path = pathlib.Path("example_files/DP_files/mitocheck_profiling_config.json")
checkpoint_path = pathlib.Path("example_files/DP_files/efficientnet-b0_weights_tf_dim_ordering_tf_kernels_autoaugment.h5")
stream.copy_DP_files(config_path, checkpoint_path)

In [8]:
stream.run_dp_stream(data_to_process, batch_size=3, start_batch=0, batch_nums=[0,1,2])

Completed: 87703K bytes transferred in 3 seconds
 (231850K bits/sec), in 1 file.
CellH5Reader initializing /home/roshankern/Desktop/Github/IDR_stream/tmp/downloads/LT0001_02/00015_01.ch5
Plate :/sample/0/plate/
Well :/sample/0/plate/LT0001_02--ex2005_11_16--sp2005_02_17--tt17--c3/experiment/
Site :/sample/0/plate/LT0001_02--ex2005_11_16--sp2005_02_17--tt17--c3/experiment/00015/position/
Parse segmentation ROIs for cell object primary__test : 0
Completed: 74464K bytes transferred in 2 seconds
 (228958K bits/sec), in 1 file.
CellH5Reader initializing /home/roshankern/Desktop/Github/IDR_stream/tmp/downloads/LT0001_02/00026_01.ch5
Plate :/sample/0/plate/
Well :/sample/0/plate/LT0001_02--ex2005_11_16--sp2005_02_17--tt17--c3/experiment/
Site :/sample/0/plate/LT0001_02--ex2005_11_16--sp2005_02_17--tt17--c3/experiment/00026/position/
Parse segmentation ROIs for cell object primary__test : 0
Completed: 58852K bytes transferred in 2 seconds
 (203748K bits/sec), in 1 file.
CellH5Reader initializi

2023-02-21 15:45:16.960186: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
Instructions for updating:
non-resource variables are not supported in the long term
/home/roshankern/Desktop/Github/IDR_stream/idrstream/DeepProfiler/deepprofiler/dataset/metadata.py:38: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  self.loadSingle(filename, delimiter, dtype)
/home/roshankern/anaconda3/envs/idrstream_dp/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(
Instructions for updating:
`normal` is a deprecated alias for `truncated_normal`
Instructions for updating:
Colocations handled automatically by placer.


2023-02-21 15:45:17,954 - WARNING - From /home/roshankern/anaconda3/envs/idrstream_dp/lib/python3.8/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
Instructions for updating:
non-resource variables are not supported in the long term
Reading metadata form ../tmp/DP_project/inputs/metadata/index.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Metadata_Plate  3 non-null      object
 1   Metadata_Well   3 non-null      object
 2   Metadata_Site   3 non-null      int64 
 3   Plate_Map_Name  3 non-null      object
 4   DNA             3 non-null      object
 5   Gene            3 non-null      object
 6   Gene_Replicate  3 non-null      int64 
dtypes: int64(2), object(5)
memory usage: 296.0+ bytes
None
{'negative 

2023-02-21 15:45:19.557792: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2023-02-21 15:45:19.580217: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-21 15:45:19.580685: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:2d:00.0 name: NVIDIA GeForce RTX 3090 computeCapability: 8.6
coreClock: 1.8GHz coreCount: 82 deviceMemorySize: 23.68GiB deviceMemoryBandwidth: 871.81GiB/s
2023-02-21 15:45:19.580705: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2023-02-21 15:45:19.591571: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2023-02-21 15:45:19.591616: I tensorflow/stream_executor/plat

2023-02-21 15:45:20,784 - WARNING - From /home/roshankern/Desktop/Github/IDR_stream/idrstream/DeepProfiler/deepprofiler/learning/profiling.py:36: The name tf.keras.backend.get_session is deprecated. Please use tf.compat.v1.keras.backend.get_session instead.

Model: "efficientnet-b0"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 112, 112, 32) 864         input[0][0]                      
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 112, 112, 32) 128         stem_conv[0][0]                  
_______________________

...

2023-02-21 15:46:48,313 - WARNING - From /home/roshankern/anaconda3/envs/idrstream_dp/lib/python3.8/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
Instructions for updating:
non-resource variables are not supported in the long term
Reading metadata form ../tmp/DP_project/inputs/metadata/index.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Metadata_Plate  3 non-null      object
 1   Metadata_Well   3 non-null      object
 2   Metadata_Site   3 non-null      int64 
 3   Plate_Map_Name  3 non-null      object
 4   DNA             3 non-null      object
 5   Gene            3 non-null      object
 6   Gene_Replicate  3 non-null      int64 
dtypes: int64(2), object(5)
memory usage: 296.0+ bytes
None
{'negative 

2023-02-21 15:46:49.915790: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2023-02-21 15:46:49.936236: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-21 15:46:49.936679: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:2d:00.0 name: NVIDIA GeForce RTX 3090 computeCapability: 8.6
coreClock: 1.8GHz coreCount: 82 deviceMemorySize: 23.68GiB deviceMemoryBandwidth: 871.81GiB/s
2023-02-21 15:46:49.936694: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2023-02-21 15:46:49.947479: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2023-02-21 15:46:49.947517: I tensorflow/stream_executor/plat

2023-02-21 15:46:51,155 - WARNING - From /home/roshankern/Desktop/Github/IDR_stream/idrstream/DeepProfiler/deepprofiler/learning/profiling.py:36: The name tf.keras.backend.get_session is deprecated. Please use tf.compat.v1.keras.backend.get_session instead.

Model: "efficientnet-b0"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 112, 112, 32) 864         input[0][0]                      
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 112, 112, 32) 128         stem_conv[0][0]                  
_______________________

/home/roshankern/anaconda3/envs/idrstream_dp/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:2424: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '
2023-02-21 15:46:51.775965: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudnn.so.8
2023-02-21 15:46:52.338716: I tensorflow/stream_executor/cuda/cuda_dnn.cc:359] Loaded cuDNN version 8401
2023-02-21 15:46:53.047695: E tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-02-21 15:46:53.047722: W tensorflow/stream_executor/gpu/asm_compiler.cc:56] Couldn't invoke ptxas --version
2023-02-21 15:46:53.048346: E tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or di

LT0001_02/352_34-1 (130 cells) : 2.18 secs
LT0001_09/15_38-1 (185 cells) : 0.52 secs
LT0001_09/26_40-1 (215 cells) : 0.60 secs
Profiling: done
